In [17]:
from utils import *

In [18]:
data_base_path = "../../../data/obrazy_testowe/"

In [19]:
distribution_data_files = os.listdir(os.path.abspath(data_base_path))

# Read distributions to data frame

In [21]:
distribution_data = []
for file in distribution_data_files:
    pgmReader = PgmReader(os.path.join(data_base_path,file))
    read_data = pgmReader.read()
    distribution_data.append([file,read_data])

In [22]:
distribution_data_df = pd.DataFrame(distribution_data,columns=["image_name","data"])

## Generate histograms test data 

In [23]:
for _, row in distribution_data_df.iterrows():
    generate_histogram(row,data_base_path)

<Figure size 640x480 with 0 Axes>

# Calcualte probabilites 

In [24]:
probabilites = []
for _, row in distribution_data_df.iterrows():
    probabilites.append(
        {
            "image_name":row["image_name"],
            "probabilities":calcualte_prob(row["data"])
        })  


# Calculate entropies

In [25]:
imgs = []
dms_entropies = []
block_entropies = []
markov_entropies = []
for img_probs in probabilites:
    prob = np.array(img_probs["probabilities"]["pixel_prob"])
    imgs.append(img_probs["image_name"])
    dms_entropies.append( simple_entropy(prob))
    block_entropies.append(block_entropy(prob))
    
    
for _ , row in distribution_data_df.iterrows():
    markov_input = np.array(row["data"]).flatten()
    markov = MarkovEntropy( markov_input)
    markov_entropies.append(markov.calculate())

## Save entropies to file

In [26]:
entropies = pd.DataFrame({"image_name":imgs,
                           "dms_entropy":dms_entropies,
                           "block_entropy":block_entropies,
                            "markov_entropy":markov_entropies
                          } )
entropies.to_csv(os.path.join(data_base_path, f"../entropies/test_images_entropies.csv") ,index=False)

In [27]:
entropies

image_name  dms_entropy  block_entropy  markov_entropy
0      barbara.pgm     0.954015       1.908030        0.724725
1         boat.pgm     0.898921       1.797843        0.662509
2  chronometer.pgm     0.764166       1.528332        0.460904
3         lena.pgm     0.930633       1.861265        0.610724
4      mandril.pgm     0.911569       1.823137        0.707182
5      peppers.pgm     0.845303       1.690606        0.431196

In [30]:


def avg_code_length(probs_with_length:dict):
    '''
    {
        "symbol":(prob,length]
    }
    '''
    result = 0 
    for prob, length in probs_with_length.values():
        result+= prob*length
    return result


2.225806451612903